## 导入所需的包

In [1]:
!git log

commit f814661261860e419b5af1b5a6c8ecadb19ea70e (HEAD -> master, origin/master, origin/HEAD)
Author: poteman <946691288@qq.com>
Date:   Mon Nov 15 10:35:34 2021 +0800

    debug: 时序xgb模型

commit 240b35118c573b6810db4b05558c0034d524c294
Author: poteman <946691288@qq.com>
Date:   Sun Nov 14 19:59:02 2021 +0800

    时序模型: 时间单位(间隔)为week.

commit 9a272d89994102b64db956a6777d1f8a7a2f8c61
Author: poteman <946691288@qq.com>
Date:   Sun Nov 14 19:41:22 2021 +0800

    时序lag和ewm特征优化.

commit 73578947ba07a64408dd0e13e9b43b9b49ba44bd
Author: poteman <946691288@qq.com>
Date:   Sun Nov 14 19:26:57 2021 +0800

    时序数据: 周期为week的数据集特征优化.

commit 85c96e3f5053aaa959bfce339779d8c15c27f372
Author: poteman <946691288@qq.com>
Date:   Fri Nov 12 17:46:09 2021 +0800

    debug: 拼表.

commit 7a30d642b74eb164a5189826bef3011a414b96c2
Author: poteman <946691288@qq.com>
Date:   Fri Nov 12 16:49:11 2021 +0800

    增加Walmart Recruiting结果.

commit 7c86e33f43a0547a927b79c3f88ef18b63c4128b
Author: poteman <946691288@qq.

In [2]:
from autox import AutoX
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## 配置数据信息

In [3]:
# 选择数据集
data_name = 'Rossmann'
path = f'../data/{data_name}'

In [4]:
# 数据表对应的数据列特征类型
feature_type = {
    "train.csv": {
        "Store": "cat",
        "DayOfWeek": "num",
        "Date": "datetime",
        "Sales": "num",
        "Open": "cat",
        "Promo": "num",
        "StateHoliday": "cat",
        "SchoolHoliday": "cat",
    },
    "test.csv": {
        "Store": "cat",
        "DayOfWeek": "num",
        "Date": "datetime",
        "Open": "cat",
        "Promo": "num",
        "StateHoliday": "cat",
        "SchoolHoliday": "cat",
    },
    "store.csv": {
        "StoreType": "cat",
        "Assortment": "cat",
        "CompetitionDistance": "num",
        "CompetitionOpenSinceMonth": "num",
        "CompetitionOpenSinceYear": "num",
        "Promo2": "cat",
        "Promo2SinceWeek": "num",
        "Promo2SinceYear": "num",
        "PromoInterval": "cat"
    }
}

relations = [
    {
            "related_to_main_table": "true", # 是否为和主表的关系
            "left_entity": "train.csv",  # 左表名字
            "left_on": ["Store"],  # 左表拼表键
            "right_entity": "store.csv",  # 右表名字
            "right_on": ["Store"], # 右表拼表键
            "type": "1-1" # 左表与右表的连接关系
        },
    {
            "related_to_main_table": "true", # 是否为和主表的关系
            "left_entity": "test.csv",  # 左表名字
            "left_on": ["Store"],  # 左表拼表键
            "right_entity": "store.csv",  # 右表名字
            "right_on": ["Store"], # 右表拼表键
            "type": "1-1" # 左表与右表的连接关系
        }
]

In [5]:
autox = AutoX(target = 'Sales', train_name = 'train.csv', test_name = 'test.csv', 
               id = ['Store'], path = path, time_series=True, ts_unit='D',time_col = 'Date',
               feature_type = feature_type,  relations = relations)

   INFO ->  [+] read sample_submission.csv
   INFO ->  Memory usage of dataframe is 0.63 MB
   INFO ->  Memory usage after optimization is: 0.20 MB
   INFO ->  Decreased by 68.7%
   INFO ->  table = sample_submission.csv, shape = (41088, 2)
   INFO ->  [+] read train.csv
   INFO ->  Memory usage of dataframe is 62.09 MB
   INFO ->  Memory usage after optimization is: 12.65 MB
   INFO ->  Decreased by 79.6%
   INFO ->  table = train.csv, shape = (1017209, 8)
   INFO ->  [+] read test.csv
   INFO ->  Memory usage of dataframe is 2.19 MB
   INFO ->  Memory usage after optimization is: 0.35 MB
   INFO ->  Decreased by 83.9%
   INFO ->  table = test.csv, shape = (41088, 7)
   INFO ->  [+] read store.csv
   INFO ->  Memory usage of dataframe is 0.09 MB
   INFO ->  Memory usage after optimization is: 0.02 MB
   INFO ->  Decreased by 76.9%
   INFO ->  table = store.csv, shape = (1115, 10)


In [6]:
sub = autox.get_submit_ts()

   INFO ->  start feature engineer
   INFO ->  feature engineer: one2M
   INFO ->  featureOne2M ops: {}
   INFO ->  ignore featureOne2M
   INFO ->  feature engineer: time
   INFO ->  featureTime ops: ['Date']
100%|██████████| 1/1 [00:01<00:00,  1.64s/it]
   INFO ->  feature engineer: ShiftTS
   INFO ->  featureShiftTS ops: ['DayOfWeek', 'Sales', 'Promo', 'store.csv__CompetitionDistance', 'store.csv__CompetitionOpenSinceMonth', 'store.csv__CompetitionOpenSinceYear', 'store.csv__Promo2SinceWeek', 'store.csv__Promo2SinceYear']
   INFO ->  featureShiftTS lags: [48, 49, 50, 51, 55, 62, 69, 78, 96, 144]
8it [01:06,  8.37s/it]
   INFO ->  feature engineer: RollingStatTS
   INFO ->  featureRollingStatTS ops: ['DayOfWeek', 'Sales', 'Promo', 'store.csv__CompetitionDistance', 'store.csv__CompetitionOpenSinceMonth', 'store.csv__CompetitionOpenSinceYear', 'store.csv__Promo2SinceWeek', 'store.csv__Promo2SinceYear']
   INFO ->  featureRollingStatTS windows: [55, 62, 96]
100%|██████████| 8/8 [03:07<00

Training with validation
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.26559	valid_1's rmse: 1.1567
[200]	training's rmse: 0.504078	valid_1's rmse: 0.445289
[300]	training's rmse: 0.253803	valid_1's rmse: 0.207147
[400]	training's rmse: 0.183276	valid_1's rmse: 0.144445
[500]	training's rmse: 0.164917	valid_1's rmse: 0.131359
[600]	training's rmse: 0.156677	valid_1's rmse: 0.126991
[700]	training's rmse: 0.151641	valid_1's rmse: 0.124556
[800]	training's rmse: 0.148193	valid_1's rmse: 0.122508
[900]	training's rmse: 0.1456	valid_1's rmse: 0.121316
[1000]	training's rmse: 0.143656	valid_1's rmse: 0.120457
[1100]	training's rmse: 0.142084	valid_1's rmse: 0.119873
[1200]	training's rmse: 0.140675	valid_1's rmse: 0.119365
[1300]	training's rmse: 0.139538	valid_1's rmse: 0.118964
[1400]	training's rmse: 0.138524	valid_1's rmse: 0.118599
[1500]	training's rmse: 0.137604	valid_1's rmse: 0.118077
[1600]	training's rmse: 0.136744	valid_1's rmse: 0.117683

   INFO ->  (1017209, 304)
   INFO ->  log1p!


Training with validation
[0]	validation_0-rmse:7.61028
Will train until validation_0-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:2.77946
[200]	validation_0-rmse:1.01640
[300]	validation_0-rmse:0.37856
[400]	validation_0-rmse:0.17287
[500]	validation_0-rmse:0.12601
[600]	validation_0-rmse:0.11874
[700]	validation_0-rmse:0.11676
[800]	validation_0-rmse:0.11517
[900]	validation_0-rmse:0.11388
[1000]	validation_0-rmse:0.11293
[1100]	validation_0-rmse:0.11199
[1200]	validation_0-rmse:0.11123
[1300]	validation_0-rmse:0.11065
[1400]	validation_0-rmse:0.11012
[1500]	validation_0-rmse:0.10965
[1600]	validation_0-rmse:0.10927
[1700]	validation_0-rmse:0.10891
[1800]	validation_0-rmse:0.10860
[1900]	validation_0-rmse:0.10833
[2000]	validation_0-rmse:0.10809
[2100]	validation_0-rmse:0.10790
[2200]	validation_0-rmse:0.10772
[2300]	validation_0-rmse:0.10753
[2400]	validation_0-rmse:0.10737
[2500]	validation_0-rmse:0.10718
[2600]	validation_0-rmse:0.10704
[2700]	validation_0-rmse:0.106

   INFO ->  feature importance
   INFO ->                                          feature  feature_importance
0                                     DayOfWeek                4811
1                                          Open                3969
2                                         Promo                4888
3                                  StateHoliday                2407
4                                 SchoolHoliday                2576
..                                          ...                 ...
299   Store__store.csv__Promo2SinceYear__ewm_62                  16
300   Store__store.csv__Promo2SinceYear__ewm_69                  16
301   Store__store.csv__Promo2SinceYear__ewm_78                  29
302   Store__store.csv__Promo2SinceYear__ewm_96                  49
303  Store__store.csv__Promo2SinceYear__ewm_144                  74

[304 rows x 2 columns]


Finished in 2:18:55.933861


In [7]:
submit = pd.read_csv('../data/Rossmann/sample_submission.csv')
sub['Id'] = submit['Id']
sub = sub[['Id', 'Sales']]

In [8]:
sub

,Id,Sales
0,1,4763.204206
1,2,7060.418940
2,3,11028.952289
3,4,7365.623853
4,5,7691.519641
...,...,...
41083,41084,2837.560593
41084,41085,7801.454122
41085,41086,5683.708887
41086,41087,19690.596931


In [9]:
sub.to_csv("./sub/autox_1115_Rossmann_oneclick.csv", index = False)